In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score
from sklearn.metrics  import  recall_score
from sklearn  import  metrics
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier


In [43]:
df_train=pd.read_csv(r'C:\Users\DELL\Desktop\projects\NetworkSecurity\notebook\data\KDDTrain+.txt')
df_test=pd.read_csv(r'C:\Users\DELL\Desktop\projects\NetworkSecurity\notebook\data\KDDTest+.txt')

In [44]:
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot'
,'num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations'
,'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate'
,'srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count'
,'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate'
,'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','difficulty_level'])

In [45]:
df_train.columns = columns
df_test.columns=columns

In [46]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125972 entries, 0 to 125971
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125972 non-null  int64  
 1   protocol_type                125972 non-null  object 
 2   service                      125972 non-null  object 
 3   flag                         125972 non-null  object 
 4   src_bytes                    125972 non-null  int64  
 5   dst_bytes                    125972 non-null  int64  
 6   land                         125972 non-null  int64  
 7   wrong_fragment               125972 non-null  int64  
 8   urgent                       125972 non-null  int64  
 9   hot                          125972 non-null  int64  
 10  num_failed_logins            125972 non-null  int64  
 11  logged_in                    125972 non-null  int64  
 12  num_compromised              125972 non-null  int64  
 13 

In [47]:
df_train.head() 

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,difficulty_level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


In [48]:
df_train.shape

(125972, 43)

In [8]:
df_test.shape

(22543, 43)

In [49]:
df_train.isnull().sum()

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

In [50]:
df_train['attack'].unique()

array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy', 'perl'], dtype=object)

In [51]:
df_train['attack']=df_train['attack'].map(lambda a: 0 if a == 'normal' else 1)

In [12]:
df_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,difficulty_level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,1,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,1,20
125968,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0,21
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,0,18
125970,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1,20


In [52]:
df_test['attack']=df_test['attack'].map(lambda a: 0 if a == 'normal' else 1)

In [14]:
df_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,difficulty_level
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,1,21
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,0,21
2,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,1,15
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,1,11
4,0,tcp,http,SF,267,14515,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,0,21
22539,0,tcp,http,SF,317,938,0,0,0,0,...,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,0,21
22540,0,tcp,http,SF,54540,8314,0,0,0,2,...,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,1,15
22541,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,0,21


In [53]:
df_train.drop(['difficulty_level'],inplace=True,axis=1)

In [54]:
df_test.drop(['difficulty_level'],inplace=True,axis=1)

In [55]:
df_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,0
1,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,1
2,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,0
3,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0
4,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,1


In [56]:
scaler=StandardScaler()
def standadization(df,cols):
    for i in cols:
        arr=df[i]
        arr=np.array(arr)
        df[i]=scaler.fit_transform(arr.reshape(len(arr),1))
    return df

In [57]:
df_train_numerical=df_train.select_dtypes(include='number').columns
df_test_numerical=df_test.select_dtypes(include='number').columns

train_label=df_train['attack']
df_train.drop(['attack'],axis=1)
test_label=df_test['attack']
df_test.drop(['attack'],axis=1)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,private,REJ,0,0,0,0,0,0,...,255,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,134,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00
2,0,icmp,eco_i,SF,20,0,0,0,0,0,...,3,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,29,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71
4,0,tcp,http,SF,267,14515,0,0,0,0,...,155,255,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,tcp,smtp,SF,794,333,0,0,0,0,...,100,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00
22539,0,tcp,http,SF,317,938,0,0,0,0,...,197,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00
22540,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07
22541,0,udp,domain_u,SF,42,42,0,0,0,0,...,255,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00


In [58]:
df_train=standadization(df_train,df_train_numerical)
df_test=standadization(df_test,df_test_numerical)
df_train['attack']=train_label
df_test['attack']=test_label

In [59]:
df_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,-0.110250,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-1.035693,-1.161035,2.736839,2.367729,-0.289105,-0.639535,-0.624874,-0.387635,-0.376389,0
1,-0.110250,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-0.809862,-0.938292,-0.174419,-0.480194,-0.289105,1.608750,1.618946,-0.387635,-0.376389,1
2,-0.110250,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089487,-0.007736,-0.095076,...,1.258746,1.066393,-0.439079,-0.383106,0.066249,-0.572087,-0.602436,-0.387635,-0.345086,0
3,-0.110250,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089487,-0.007736,-0.095076,...,1.258746,1.066393,-0.439079,-0.480194,-0.289105,-0.639535,-0.624874,-0.387635,-0.376389,0
4,-0.110250,tcp,private,REJ,-0.007762,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-0.873095,-1.005115,-0.068555,-0.480194,-0.289105,-0.639535,-0.624874,2.874397,2.753901,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,-0.110250,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-0.818895,-0.938292,-0.121487,-0.480194,-0.289105,1.608750,1.618946,-0.387635,-0.376389,1
125968,-0.107178,udp,private,SF,-0.007744,-0.004883,-0.014089,-0.089487,-0.007736,-0.095076,...,1.159381,0.977296,-0.386147,-0.447832,-0.289105,-0.639535,-0.624874,-0.387635,-0.376389,0
125969,-0.110250,tcp,smtp,SF,-0.007382,-0.004823,-0.014089,-0.089487,-0.007736,-0.095076,...,-0.773729,-0.893743,-0.121487,-0.480194,-0.289105,0.979230,-0.624874,-0.355015,-0.376389,0
125970,-0.110250,tcp,klogin,S0,-0.007762,-0.004919,-0.014089,-0.089487,-0.007736,-0.095076,...,-0.972460,-1.094212,-0.174419,-0.480194,-0.289105,1.608750,1.618946,-0.387635,-0.376389,1


In [60]:
df_train['protocol_type'].unique()

array(['udp', 'tcp', 'icmp'], dtype=object)

In [26]:
service_feature_train=df_train['service'].unique()

In [27]:
df_train['flag'].nunique()

11

In [28]:
df_test['protocol_type'].unique()

array(['tcp', 'icmp', 'udp'], dtype=object)

In [29]:
df_test['flag'].unique()

array(['REJ', 'SF', 'RSTO', 'S0', 'RSTR', 'SH', 'S3', 'S2', 'S1',
       'RSTOS0', 'OTH'], dtype=object)

In [30]:
df_test['flag'].nunique()

11

In [31]:
service_feature_test=df_test['service'].unique()

In [61]:
for x in service_feature_train:
    if x not in service_feature_test:
        print(x)

red_i
urh_i
http_8001
aol
http_2784
harvest


In [62]:
df_train = df_train[df_train.service != "red_i"]
df_train = df_train[df_train.service != "urh_i"]
df_train= df_train[df_train.service != "http_8001"]
df_train = df_train[df_train.service !="aol"]
df_train = df_train[df_train.service != "http_2784"]
df_train = df_train[df_train.service != "harvest"]

In [64]:
df_train.shape

(125947, 42)

In [65]:
df_train=pd.get_dummies(df_train,columns=['protocol_type','service','flag'],prefix="",prefix_sep="")
df_test=pd.get_dummies(df_test,columns=['protocol_type','service','flag'],prefix="",prefix_sep="")

In [66]:
y_train = df_train["attack"]
X_train = df_train.drop("attack", axis=1)
print('X_train shape:',X_train.shape,'\ny_train shape:',y_train.shape)

X_train shape: (125947, 116) 
y_train shape: (125947,)


In [67]:
y_test = df_test["attack"]
X_test = df_test.drop("attack", axis=1)
print('X_test shape:',X_test.shape,'\ny_train has shape:',y_test.shape)

X_test shape: (22543, 116) 
y_train has shape: (22543,)


In [68]:
selected_features=SelectKBest(mutual_info_classif,k=50).fit(X_train,y_train)
features=X_train.columns[selected_features.get_support()]


In [69]:
X_train=X_train[features]
X_test=X_test[features]

In [70]:
def printScore(expected, predicted):
    accuracy = accuracy_score(expected, predicted)
    recall = recall_score(expected, predicted, average='micro')
    precision = precision_score(expected, predicted , average='micro')
    f1 = f1_score(expected, predicted , average='micro')
    return accuracy, recall, precision, f1

In [71]:
models={
    
    'RandomForest': RandomForestClassifier(),
    'AdaBoost':AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'SupportVector': SVC(),
    'KNN':KNeighborsClassifier(),
    'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis(),
    'GaussianNaiveBayes':GaussianNB(),
    "XGBClassifier": XGBClassifier(),
    'GradientBoosting':GradientBoostingClassifier()

    
}
model_list = []
accuracy_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
     # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    model_train_accuracy , model_train_recall, model_train_precision,model_train_f1 = printScore(y_train, y_train_pred)
    model_test_accuracy , model_test_recall, model_test_precision,model_test_f1 = printScore(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Acurracy Score: {:.4f}".format(model_train_accuracy))
    print("- Recall score: {:.4f}".format(model_train_recall))
    print("- Precision score: {:.4f}".format(model_train_precision))
    print("- F1 score: {:.4f}".format(model_train_f1))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Acurracy Score: {:.4f}".format(model_test_accuracy))
    print("- Recall score: {:.4f}".format(model_test_recall))
    print("- Precision score: {:.4f}".format(model_test_precision))
    print("- F1 score: {:.4f}".format(model_test_f1))
    
    accuracy_list.append(model_test_accuracy)
    
    print('='*35)
    print('\n')

RandomForest
Model performance for Training set
- Acurracy Score: 0.9999
- Recall score: 0.9999
- Precision score: 0.9999
- F1 score: 0.9999
----------------------------------
Model performance for Test set
- Acurracy Score: 0.7512
- Recall score: 0.7512
- Precision score: 0.7512
- F1 score: 0.7512


AdaBoost
Model performance for Training set
- Acurracy Score: 0.9916
- Recall score: 0.9916
- Precision score: 0.9916
- F1 score: 0.9916
----------------------------------
Model performance for Test set
- Acurracy Score: 0.7700
- Recall score: 0.7700
- Precision score: 0.7700
- F1 score: 0.7700


DecisionTree
Model performance for Training set
- Acurracy Score: 0.9999
- Recall score: 0.9999
- Precision score: 0.9999
- F1 score: 0.9999
----------------------------------
Model performance for Test set
- Acurracy Score: 0.8140
- Recall score: 0.8140
- Precision score: 0.8140
- F1 score: 0.8140


SupportVector
Model performance for Training set
- Acurracy Score: 0.9945
- Recall score: 0.9945
-

AttributeError: 'Flags' object has no attribute 'c_contiguous'

In [ ]:
models={
                        
    'RandomForest':(RandomForestClassifier(),
                    {'n_estimators': [100, 500, 1000],
                     'max_depth': [None, 5,10, 20],
                      'max_features': ['sqrt', 'log2'],
                    }),
    'AdaBoost': (AdaBoostClassifier(),
                 {'learning_rate': [0.01, 0.1, 1],
                  'n_estimators': [50, 100, 200],
                }),
    'DecisionTree': (DecisionTreeClassifier(),
                     {'max_depth': [None,2,4,6,8, 10, 15],
                      'min_samples_split': [2, 4,6,8,10],
                      'criterion': ['gini', 'entropy']}),
    'SupportVector': (SVC(),
                      {'C': [0.1, 1, 10],
                        'gamma': ['scale', 'auto', 0.1, 1,5]}),
    'KNN':(KNeighborsClassifier(),
           {'n_neighbors': np.arange(1, 25),
              'weights': ['uniform', 'distance']
            
           }),
    "XGBClassifier": (XGBClassifier(),
                      {'learning_rate': [0.01, 0.1, 0.2],
                       'max_depth': [None,3, 5, 7,10,50],
                       'min_child_weight': [1, 3, 5,10,50,100],
      }),
    'GradientBoosting':(GradientBoostingClassifier(),
                        { 'n_estimators': [50, 100, 200],
                          'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [None,3, 5, 7],
                         
                        })
}

best_models = {}

for model_name, (model, param_grid) in models.items():
    grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    best_models[model_name] = grid_search.best_estimator_



best_model_name = None
best_accuracy = 0

for model_name, best_model in best_models.items():
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f'{model_name}: Accuracy = {accuracy:.4f}')
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = model_name

print(f'Best Model: {best_model_name}')
